In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/llm_finetuning/llm_finetuning/notebooks'
curr_filename = "vanilla_rte_baseline"


Mounted at /content/drive
/content/drive/MyDrive/llm_finetuning/llm_finetuning/notebooks


In [2]:
!pip install -q transformers accelerate bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments, AdamW
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pdb
import torch
import warnings
warnings.simplefilter("ignore")

# Set seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Load RTE dataset
rte_dataset = load_dataset("glue", "rte")

# Load HANS dataset
hans_data = load_dataset("hans", split="validation"
)

# Define tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Preprocess RTE data
def preprocess_rte(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

rte_dataset = rte_dataset.map(preprocess_rte, batched=True)

# Preprocess HANS data
def preprocess_hans(examples):
    return tokenizer(
        examples["premise"], examples["hypothesis"],  # Premise and hypothesis
        truncation=True, padding="max_length", max_length=128
    )

# Apply preprocessing
hans_dataset = hans_data.map(preprocess_hans, batched=True)

# Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Few-shot training
few_shot_sample_size = [2, 32, 128]
num_epochs = 40
batch_size = 32
learning_rate = 1e-5
num_runs = 10

results_df = pd.DataFrame(columns=["n", "run", "in_domain_accuracy", "out_of_domain_accuracy"])

for run_idx in range(num_runs):
  for n in few_shot_sample_size:
        # Reinitialize the model
        model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # Select n examples per class from RTE training data
        entailment_indices = np.where(np.array(rte_dataset["train"]["label"]) == 1)[0]
        non_entailment_indices = np.where(np.array(rte_dataset["train"]["label"]) == 0)[0]
        entailment_indices = np.random.choice(entailment_indices, n, replace=False)
        non_entailment_indices = np.random.choice(non_entailment_indices, n, replace=False)
        indices = np.concatenate([entailment_indices, non_entailment_indices])

        train_dataset = rte_dataset["train"].select(indices)

        # Training arguments
        training_args = TrainingArguments(
            output_dir="./results",
            overwrite_output_dir=True,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            learning_rate=learning_rate,
            weight_decay=0.0,
            save_steps=10_000,
            save_total_limit=2
        )

        # Trainer setup
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            compute_metrics=compute_metrics,
            optimizers=(optimizer, None),
        )

        # Train the model
        trainer.train()

        # Evaluate in-domain (RTE)
        print(f"Evaluating in-domain performance for n={n}...")
        in_domain_results = trainer.evaluate(eval_dataset=rte_dataset["validation"])
        in_domain_accuracy = in_domain_results["eval_accuracy"]

        # Evaluate out-of-domain (HANS)
        print(f"Evaluating out-of-domain performance for n={n}...")
        hans_results = trainer.evaluate(eval_dataset=hans_dataset)
        out_of_domain_accuracy = hans_results["eval_accuracy"]

        print(f"n={n}, run={run_idx}, In-domain Accuracy: {in_domain_accuracy}, Out-of-domain Accuracy: {out_of_domain_accuracy}")

        # Save results
        new_row = pd.DataFrame({
            "n": [n],
            "run": [run_idx],
            "in_domain_accuracy": [in_domain_accuracy],
            "out_of_domain_accuracy": [out_of_domain_accuracy],
        })

        results_df = pd.concat([results_df, new_row], ignore_index=True)




README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

...

Evaluating out-of-domain performance for n=32...
n=32, run=9, In-domain Accuracy: 0.5631768953068592, Out-of-domain Accuracy: 0.4961333333333333


Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Evaluating in-domain performance for n=128...


Evaluating out-of-domain performance for n=128...
n=128, run=9, In-domain Accuracy: 0.6209386281588448, Out-of-domain Accuracy: 0.49693333333333334


In [8]:
# Save results to a CSV file
results_df.to_csv(curr_filename + ".csv", index=False)

In [5]:

print(results_df)

      n run  in_domain_accuracy  out_of_domain_accuracy
0     2   0            0.541516                0.497933
1    32   0            0.563177                0.496133
2   128   0            0.620939                0.496933
3     2   1            0.505415                0.508167
4    32   1            0.563177                0.496133
5   128   1            0.620939                0.496933
6     2   2            0.505415                0.508167
7    32   2            0.563177                0.496133
8   128   2            0.620939                0.496933
9     2   3            0.505415                0.508167
10   32   3            0.563177                0.496133
11  128   3            0.620939                0.496933
12    2   4            0.505415                0.508167
13   32   4            0.563177                0.496133
14  128   4            0.620939                0.496933
15    2   5            0.505415                0.508167
16   32   5            0.563177                0

In [7]:
results_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>n</th>\n      <th>run</th>\n      <th>in_domain_accuracy</th>\n      <th>out_of_domain_accuracy</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>2</td>\n      <td>0</td>\n      <td>0.541516</td>\n      <td>0.497933</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>32</td>\n      <td>0</td>\n      <td>0.563177</td>\n      <td>0.496133</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>128</td>\n      <td>0</td>\n      <td>0.620939</td>\n      <td>0.496933</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>2</td>\n      <td>1</td>\n      <td>0.505415</td>\n      <td>0.508167</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>32</td>\n      <td>1</td>\n      <td>0.563177</td>\n      <td>0.496133</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>128</td>\n      <td>1</td>\n      <td>0.620939</td>\n      <td>0.496933</td>\n    <